In [2]:
import pandas as pd
import MeCab
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
# 分かち書きの中で使うオブジェクト生成
tagger = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
# ひらがなのみの文字列にマッチする正規表現
kana_re = re.compile("^[ぁ-ゖ]+$")


def mecab_tokenizer(text):
    # テキストを分かち書きする関数を準備する
    parsed_lines = tagger.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]

    # 名詞,動詞,形容詞のみに絞り込み
    target_pos = ["名詞", "動詞", "形容詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]
    # ひらがなのみの単語を除く
    token_list = [t for t in token_list if not kana_re.match(t)]
    # 数値を含む単語も除く
    token_list = [t for t in token_list if not re.match("\d", t)]
    return " ".join(token_list)


In [92]:
# def calc_vectors(utterances, n_components):
#     tokenized_utterances = [mecab_tokenizer(utt) for utt in utterances]
#     # テキストデータをBOW形式に変換する
#     tf_vectorizer = CountVectorizer(
#         token_pattern='(?u)\\b\\w+\\b',
# #         max_df=0.90,
# #         min_df=10,
#     )
#     tf = tf_vectorizer.fit_transform(tokenized_utterances)
#     # LDAのモデル作成と学習
#     lda = LatentDirichletAllocation(n_components=n_components)
#     return lda.fit_transform(tf)

In [93]:
vectors = calc_vectors(
    [' これから本番の収録を開始します。', ' はじめまして・・私の名前は「メイ」です。「対話の実験」の手伝いをしています。今日はよろしくお願いします。', 'よろしくお願いします', ' きょうはここまでなにで来られましたか?乗り物は乗りましたか?', 'そうですね 電車で 来て歩いて来ました', ' ここに来るまで迷いませんでしたか・・', 'ここに来るまでは人に教えてもらったので 迷いませんでした', ' 大学の中でも、たてものが奥にあるので、みなさん来るのが「たいへん」でしたとよく言われます・・', '(F あー)そうなんですか 確かに奥にありました', ' きょうは来ていただいてありがとうございます!', ' それでは「食べ物」について話しましょう!', '(F はい)', ' 甘いものはお好きですか?', '甘いものは洋菓子じゃなくて和菓子のほうが好きですね', ' 普段はどこであまいものを買いますか?', '普段はいやあコンビニが多いですね', ' 和菓子はどんなものですか'],
    n_components=2
)

In [94]:
vectors

array([[0.13091425, 0.86908575],
       [0.05793763, 0.94206237],
       [0.25567502, 0.74432498],
       [0.86907345, 0.13092655],
       [0.89724667, 0.10275333],
       [0.83040325, 0.16959675],
       [0.89712678, 0.10287322],
       [0.202934  , 0.797066  ],
       [0.12892683, 0.87107317],
       [0.74627338, 0.25372662],
       [0.82643011, 0.17356989],
       [0.25576428, 0.74423572],
       [0.83002433, 0.16997567],
       [0.89749754, 0.10250246],
       [0.17251835, 0.82748165],
       [0.12985237, 0.87014763],
       [0.74505098, 0.25494902]])

In [125]:
from scipy.spatial import distance
import MeCab
import numpy as np

# def calc_similarity(topic1, topic2, window_size=5):
def calc_similarity(topic1, topic2, n_components=5):
    # テキストデータをBOW形式に変換する
    tf_vectorizer = CountVectorizer(
        token_pattern='(?u)\\b\\w+\\b',
#         max_df=0.90,
#         min_df=10,
    )
    lda_vectorizer = LatentDirichletAllocation(n_components=n_components)
    
    def train(utterances):
        tokenized_utterances = [mecab_tokenizer(utt) for utt in utterances]
        tf = tf_vectorizer.fit_transform(tokenized_utterances)
        # LDAの学習
        lda_vectorizer.fit(tf)
    
    def predict(utterance):
        tokenized_utterance = mecab_tokenizer(utterance)
        tf = tf_vectorizer.transform([tokenized_utterance])
        return lda_vectorizer.transform(tf)
    
    train(topic1 + topic2)
    
    vec1 = predict(' '.join(topic1))
    vec2 = predict(' '.join(topic2))
    
#     vectors = calc_vectors(topic1 + topic2, n_components=n_components)
#     n1 = len(topic1)
#     vec1 = np.mean(vectors[:n1], axis=0)
#     vec2 = np.mean(vectors[n1:], axis=0)

#     vec1 = np.mean(vectors[n1-window_size:n1], axis=0)
#     vec2 = np.mean(vectors[n1:n1+window_size], axis=0)
    return 1 - distance.cosine(vec1, vec2)

In [126]:
calc_similarity(['コンサートとかには行きますか？'], ['コンサートは行かないですけど、映画とかは好きですね'])

0.1516435019040483

In [127]:
calc_similarity(['コンサートとかには行きますか？'], ['コンサートは行かないですけど、映画とかは好きですね', '映画といえば、閃光のハサウェイは見ましたか？'])

0.8133555761682684

In [128]:
calc_similarity(['コンサートとかには行きますか？'], ['織田信長の野望'])

0.20015429879330116

In [129]:
calc_similarity(['豊臣秀吉の狙いはなんだったのか？'], ['織田信長の野望'])

0.20015429879323132

In [207]:
def topic_generator(text_gen, threshold=0.3, max_utterances=50, min_utterances=10, delay=3, n_components=5):
    cur_topic = [next(text_gen)]
    
    for text in text_gen:
        similarity = calc_similarity(cur_topic[:-delay], cur_topic[-delay:] + [text], n_components=n_components)
        if (
            similarity < threshold
            or len(cur_topic) >= max_utterances
        ) and len(cur_topic) >= min_utterances + delay:
            yield cur_topic[:-delay]
            cur_topic = cur_topic[-delay:] + [text]
        else:
            cur_topic.append(text)

In [208]:
import jsonlines
import unicodedata
import re

def transcript_generator(path):
    with jsonlines.open(path) as reader:
        for data in reader:
            yield preprocess_transcript(data['transcript'])

def preprocess_transcript(transcript):
    transcript = unicodedata.normalize('NFKC', transcript)
    transcript = transcript.replace('|', ' ')
    transcript = re.sub(r'\(.+\)', '', transcript)
    return transcript

In [229]:
topic_gen = topic_generator(
    transcript_generator('data/golden_transcripts/1911F2002.jsonl'),
    threshold=0.2,
    max_utterances=50,
    min_utterances=10,
    delay=3,
    n_components=5
)

for topic in topic_gen:
    print(topic, '\n')
    

[' これから本番の収録を開始します。', ' はじめまして・・私の名前は「メイ」です。「対話の実験」の手伝いをしています。今日はよろしくお願いします。', 'よろしくお願いします', ' きょうはここまでなにで来られましたか?乗り物は乗りましたか?', 'そうですね 電車で 来て歩いて来ました', ' ここに来るまで迷いませんでしたか・・', 'ここに来るまでは人に教えてもらったので 迷いませんでした', ' 大学の中でも、たてものが奥にあるので、みなさん来るのが「たいへん」でしたとよく言われます・・', 'そうなんですか 確かに奥にありました', ' きょうは来ていただいてありがとうございます!', ' それでは「食べ物」について話しましょう!'] 

['', ' 甘いものはお好きですか?', '甘いものは洋菓子じゃなくて和菓子のほうが好きですね', ' 普段はどこであまいものを買いますか?', '普段はいやあコンビニが多いですね', ' 和菓子はどんなものですか', '和菓子どんなもの 大福とかみたらし団子とか', ' 大福、ですか?', 'はい', ' コンビニですか。よく、行くのですか。'] 

['コンビニよく行きますね', ' そうなんですね、ちょっと特別なおかしを買うときは近くのお店とか「デパートの地下」とか、利用しますか?', '利用しますね', ' デパートのホームページを見て、キラキラしていて楽しそうと思うのですが、実際はいかがですか?', '実際は人が多くてしんどい所ですね', ' そうなんですか.', ' その中でもおすすめのスイーツはありますか?', '', ' そうなんですか.', 'うん', ' 私の友人はあんこ系が好きだそうです。あずきときなこは、体や肌にいいそうで、よく食べてるそうです・・'] 

['', ' おばあちゃんがよく家で出してくれていたと言っていました。', 'いいおばあちゃんですね', ' いい思い出だったようです。', '', ' からい食べものとはどんな食べものが好きですか?', '辛い食べ物は  なんか麺も好きやしご飯丼ぶり系も好きです', ' 塩辛いほうですか、トウガラシのからさとかですか?', ' 塩辛いほうですね', ' 具体的な食べ物の名前を教えてください。', '具体的な  オムライスと 牛丼と 

In [127]:
a = [' これから本番の収録を開始します。', ' はじめまして・・私の名前は「メイ」です。「対話の実験」の手伝いをしています。今日はよろしくお願いします。', 'よろしくお願いします', ' きょうはここまでなにで来られましたか?乗り物は乗りましたか?', 'そうですね 電車で 来て歩いて来ました', ' ここに来るまで迷いませんでしたか・・', 'ここに来るまでは人に教えてもらったので 迷いませんでした', ' 大学の中でも、たてものが奥にあるので、みなさん来るのが「たいへん」でしたとよく言われます・・', '(F あー)そうなんですか 確かに奥にありました', ' きょうは来ていただいてありがとうございます!', ' それでは「食べ物」について話しましょう!', '(F はい)', ' 甘いものはお好きですか?', '甘いものは洋菓子じゃなくて和菓子のほうが好きですね', ' 普段はどこであまいものを買いますか?', '普段はいやあコンビニが多いですね', ' 和菓子はどんなものですか']
b = ['和菓子どんなもの 大福とかみたらし団子とか', ' 大福、ですか?', 'はい', ' コンビニですか。よく、行くのですか。', 'コンビニよく行きますね', ' そうなんですね、ちょっと特別なおかしを買うときは近くのお店とか「デパートの地下」とか、利用しますか?', '(F あー)利用しますね', ' デパートのホームページを見て、キラキラしていて楽しそうと思うのですが、実際はいかがですか?', '実際は人が多くてしんどい所ですね', ' そうなんですか.', ' その中でもおすすめのスイーツはありますか?', '(F えー) は なんかケーキ (D わか)', ' そうなんですか.', 'うん', ' 私の友人はあんこ系が好きだそうです。あずきときなこは、体や肌にいいそうで、よく食べてるそうです・・', '(F ふーん) (F へー)', ' おばあちゃんがよく家で出してくれていたと言っていました。', '(F えー)いいおばあちゃんですね', ' いい思い出だったようです。', '(F はあ)', ' からい食べものとはどんな食べものが好きですか?', '辛い食べ物は (F うーん) なんか麺も好きやしご飯丼ぶり系も好きです', ' 塩辛いほうですか、トウガラシのからさとかですか?', '(D し) 塩辛いほうですね', ' 具体的な食べ物の名前を教えてください。', '具体的な (F えー) オムライスと 牛丼と なんかそんな感じの', ' 牛丼、ですか?', 'はい', ' それは「最近」食べられたんですか?', '最近は食べてないです', ' そうなんですね、お店ですか、ご自宅ですか?', 'お店ですね', ' なるほど、そうなんですね、', '(F はい)', ' オムライスそれは「最近」食べられたんですか?', 'オムライスは食べました', ' オムライス、どこで召し上がったのですか・・', 'オムライスは家で食べました', ' そうですか、それは美味しそうですね・・', ' お友達とランチに行ったりしますか?', 'よく行きますね', ' なるほど、どんなところに行きますか?和食ですか洋食ですか?それともはやりのカフェとかですか?', '(D よう) 流行りのカフェですね', ' 例えば?', '例えば あれどこやろ 梅田にある ような 感じ', ' うめだ、ですか?', ' 、ですか?']
calc_similarity('\n'.join(a[:2]), '\n'.join(b[:3]))

0.6496319770812988

In [49]:
list(gen)

[2, 3]

In [83]:
[1,2,3,4][:-2]

[1, 2]